Xây dựng bộ phát hiện đối tượng với thuật toán YOLOv3 được viết bằng thư viện Keras. Thư viện này bao gồm đầy đủ các module: phát hiện đối tượng với pre-trainded model, huấn luyện lại mô hình, phát hiện đối tượng ảnh trên ảnh/video/webcam.

Đầu tiên, ta sẽ khởi tạo Google Colab với sử dụng 1 GPU. Sau đó thực hiện kết nối với Google Drive của tài khoản cá nhân như sau:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
lab_path = "/content/gdrive/My\ Drive/AdvancedComputerVision/keras-yolo3"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Sau đó, sử dụng git để clone thư mục project từ github của tác giả experiencor với đường dẫn như sau. Lưu ý rằng, ta nên đặt thực project kế bên file jupyter notebook này để tiện thao tác.

In [ ]:
#!git clone https://github.com/experiencor/keras-yolo3.git $lab_path

Sau đó tiến hành cài đặt các thư viện cần thiết để có thể sử dụng được keras-yolo3. Đây là bước thực hiện quan trọng vì thông thường các thư viện sẽ sử dụng các thư viện bên thứ 3 với các phiên bản cố định. Do đó, để đảm bảo thư viện keras-yolo3 sử dụng trơn tru thì ta sẽ cài đặt các thư viện kèm theo trong file requirements.txt.

In [ ]:
!pip install -r $lab_path/requirements.txt

     |████████████████████████████████| 61kB 2.5MB/s 
     |████████████████████████████████| 2.4MB 7.6MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 92kB 7.9MB/s 
     |████████████████████████████████| 20.2MB 1.5MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 1.3MB 41.7MB/s 
     |████████████████████████████████| 276kB 41.5MB/s 
     |████████████████████████████████| 3.8MB 37.9MB/s 
     |████████████████████████████████| 412.3MB 34kB/s 
     |████████████████████████████████| 512kB 31.1MB/s 
     |████████████████████████████████| 327kB 39.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=ad613cb165f029f76d7ae84cb4b8d88a1cddb0ca83230b7f69b894ac14887a3e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Created wheel for opt-einsum: filename=opt_einsum-3.1.0-cp36-none-any.whl size=616

1. Thử nghiệm với pretrained model của tác giả thuật toán YOLO.

File mô hình huấn luyện sẵn được đặt ở đường dẫn sau https://pjreddie.com/media/files/yolov3.weights
Lưu ý là file mô hình cần được đặt trong thư mục gốc của thư viện để tiện cho việc thực hiện.

In [ ]:
#!wget https://pjreddie.com/media/files/yolov3.weights -O $lab_path/yolov3.weights"

Sau đó, ta tiến hành sử dụng pretrained model để phát hiện đối tượng trong ảnh cho trước. Sử dụng chương trình yolo3_one_file_to_detect_them_all.py với file mô hình đã download ở file trước đó để phát hiện các đối tượng trong ảnh. Tập dữ liệu các đối tượng phát hiện được lấy từ tập PASCAL VOC 2012.

Ảnh kết quả phát hiện đối tượng trên pretrained model cho tập dữ liệu này được minh hoạ như ở hình sau:

![alt text](https://i.imgur.com/6nun5fl.jpg)

In [ ]:
!python $lab_path/yolo3_one_file_to_detect_them_all.py -w $lab_path/yolov3.weights -i $lab_path/dog-tales-img.jpg

2. Huấn luyện dữ liệu với keras-yolo3

![alt text](https://i.imgur.com/45SRniL.jpg)
Bước tiếp theo, ta sẽ tiến hành huấn luyện trên tập dữ liệu mới là gấu mèo (raccoon). Tập dữ liệu này được tải từ https://github.com/experiencor/raccoon_dataset.git

Lưu ý rằng, ta sẽ chỉ sử dụng 2 thư mục của github này là images và annotations. Trong đó,

*   images: thư mục chứa các ảnh về gấu mèo
*   annotations: tương ứng với từng file ảnh của thư mục images, ta có một file .xml chứa thông tin về vị trí chính xác của đối tượng (gấu mèo) trong ảnh.

Các bạn nên xem kỹ định dạng của các file XML này để biết cấu trúc file gán nhãn phục vụ cho việc thực hiện đồ án sau này.



In [ ]:
#!git clone https://github.com/experiencor/raccoon_dataset.git $lab_path/raccoon_dataset

Một file khác cũng rất quan trọng của các mô hình huấn luyện học sâu nói chung và YOLO nói riêng là file cấu hình (config) các tham số khi huấn luyện.

Trong thư viện này, file config được đặt trong thư mục /zoo. Ví dụ như config_raccoon.json. Các tham số cơ bản của file config như:
- Đường dẫn đến thư mục train, validation (mặc định chia tỉ lệ train:validation = 8:2 nếu không có tham số gắn sẵn)
- GPU sử dụng để huấn luyện
- File pretrained cho tập image net backend.h5
- Learning rate
- Batch size

Sau khi đã cấu hình file config, ta sẽ tiến hành huấn luyện:

In [ ]:
!python $lab_path/train.py -c $lab_path/zoo/config_raccoon.json

Using TensorFlow backend.
2020-07-15 11:52:19.116907: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-07-15 11:52:19.122707: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-07-15 11:52:19.123066: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1590bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-07-15 11:52:19.123100: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-07-15 11:52:19.124985: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-15 11:52:19.233778: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07

Sau khi quá trình huấn luyện kết thúc, hệ thống sẽ lưu mô hình raccoon.h5 trong thư mục của máy ảo colab. Ta có thể sửa lại mã nguồn của thư viện để các file mô hình và file trung gian đặt trong thư mục của thư viện cho gọn gàng.

Sau đó, ta sẽ tiến hành thử nghiệm mô hình đã huấn luyện trên ảnh mẫu lấy từ mạng internet hoặc thư mục ảnh validation/train.

In [ ]:
!python $lab_path/predict.py -c $lab_path/zoo/config_raccoon.json -i '/content/Bernese-Mountain-Dog-On-White-01.jpg'

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-07-15 12:57:47.701296: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-15 12:57:47.722474: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-15 12:57:47.723343: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-07-15 12:57:47.723909: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-07-15 12:57:47.730666: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-07-15 12:57:47.73427

Như vậy, trong phần bài tập này chúng ta đã làm quen với thư viện keras-yolo3. Trong phần đồ án giữa kỳ, ta sẽ tiến hành trên dữ liệu video giao thông được thu tại TP. Hồ Chí Minh. Ví dụ, ảnh sau được cắt tại góc đường Nam Kỳ Khởi Nghĩa - Võ Thị Sáu.

![alt text](https://i.imgur.com/NQBeNpG.jpg)

Mục tiêu của đồ án này là sử dụng thuật toán YOLOv3 để tiến hành ước lượng lưu lượng giao thông theo các hướng đi.

Đầu tiên, ta sẽ thử sử dụng pretrained model lên dữ liệu video thực tế.

Sau đó tiến hành xây dựng bộ dữ liệu gán nhãn cho ngữ cảnh camera giao thông.

Huấn luyện lại mô hình với dữ liệu gán nhãn mới.

Phát hiện và đếm các đối tượng xe máy, xe hơi trong video giao thông.

